In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Vehicle_insurance.csv')
df.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,...,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,...,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,...,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,...,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,...,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Month                 15420 non-null  object
 1   WeekOfMonth           15420 non-null  int64 
 2   DayOfWeek             15420 non-null  object
 3   Make                  15420 non-null  object
 4   AccidentArea          15420 non-null  object
 5   DayOfWeekClaimed      15420 non-null  object
 6   MonthClaimed          15420 non-null  object
 7   WeekOfMonthClaimed    15420 non-null  int64 
 8   Sex                   15420 non-null  object
 9   MaritalStatus         15420 non-null  object
 10  Age                   15420 non-null  int64 
 11  Fault                 15420 non-null  object
 12  PolicyType            15420 non-null  object
 13  VehicleCategory       15420 non-null  object
 14  VehiclePrice          15420 non-null  object
 15  FraudFound_P          15420 non-null

In [4]:
# non-ordered categorical features

non_ordered_features = ['Make', 'PolicyType', 'VehicleCategory', 'PolicyNumber', 'BasePolicy', 'MaritalStatus']

for i in non_ordered_features:
    unique_values = df[i].unique()
    total = len(unique_values)
    print("=============================================================")
    print(f"Unique values in '{i}' : {unique_values}")
    print("-------------------------------------------------------------")
    print(f"total number of unique values in '{i}': {total}")

    

Unique values in 'Make' : ['Honda' 'Toyota' 'Ford' 'Mazda' 'Chevrolet' 'Pontiac' 'Accura' 'Dodge'
 'Mercury' 'Jaguar' 'Nisson' 'VW' 'Saab' 'Saturn' 'Porche' 'BMW' 'Mecedes'
 'Ferrari' 'Lexus']
-------------------------------------------------------------
total number of unique values in 'Make': 19
Unique values in 'PolicyType' : ['Sport - Liability' 'Sport - Collision' 'Sedan - Liability'
 'Utility - All Perils' 'Sedan - All Perils' 'Sedan - Collision'
 'Utility - Collision' 'Utility - Liability' 'Sport - All Perils']
-------------------------------------------------------------
total number of unique values in 'PolicyType': 9
Unique values in 'VehicleCategory' : ['Sport' 'Utility' 'Sedan']
-------------------------------------------------------------
total number of unique values in 'VehicleCategory': 3
Unique values in 'PolicyNumber' : [    1     2     3 ... 15418 15419 15420]
-------------------------------------------------------------
total number of unique values in 'PolicyNumber

### Features to remove:
1. 'PolicyNumber'
2. 'PolicyType'

### Features that need to be encoded:
1. Ordered/ cyclic : Features related to Month, DayOfWeek...
- Cyclic Encoding (Sin or Cosine transformation)

2. Non-ordered: 
- Label encoding : AccidentArea, Sex, PolicyReportfilled, witness, agentType
- One hot encoding: Make, MaritalStatus, VehicleCategory, BasePolicy
 

In [5]:
# remove useless features:

df_new = df.drop(columns=['PolicyNumber', 'PolicyType'])

In [6]:
# Added
Day_month_claimed = ['DayOfWeekClaimed', 'MonthClaimed']

for i in Day_month_claimed:
    unique_values = df[i].unique()
    total = len(unique_values) 
    print("=============================================================")
    print(f"Unique values in '{i}' : {unique_values}")
    print("-------------------------------------------------------------")
    print(f"total number of unique values in '{i}': {total}")

Unique values in 'DayOfWeekClaimed' : ['Tuesday' 'Monday' 'Thursday' 'Friday' 'Wednesday' 'Saturday' 'Sunday'
 '0']
-------------------------------------------------------------
total number of unique values in 'DayOfWeekClaimed': 8
Unique values in 'MonthClaimed' : ['Jan' 'Nov' 'Jul' 'Feb' 'Mar' 'Dec' 'Apr' 'Aug' 'May' 'Jun' 'Sep' 'Oct'
 '0']
-------------------------------------------------------------
total number of unique values in 'MonthClaimed': 13


In [7]:
# cyclic encoding:

Month_features = ['Month', 'MonthClaimed'] # Jan ~ Dec
dayInWeeek_features = ['DayOfWeek', 'DayOfWeekClaimed'] # Mon~Fri
weekOfMonth_features = ['WeekOfMonth', 'WeekOfMonthClaimed'] # 1~5

# transform to number

# Mapping dictionaries
month_to_num = {'0':0, 'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

day_to_num = {'0': 0, 'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday':6, 'Sunday':7}

# Convert month and day columns to numerical format
df_new['Month'] = df_new['Month'].map(month_to_num)
df_new['MonthClaimed'] = df_new['MonthClaimed'].map(month_to_num)
df_new['DayOfWeek'] = df_new['DayOfWeek'].map(day_to_num)
df_new['DayOfWeekClaimed'] = df_new['DayOfWeekClaimed'].map(day_to_num)

In [8]:
# Apply cyclical encoding for months and days of the week
for column in ['Month', 'MonthClaimed']:
    df_new[column + '_sin'] = np.sin(2 * np.pi * df_new[column] / 12)
    df_new[column + '_cos'] = np.cos(2 * np.pi * df_new[column] / 12)

for column in ['DayOfWeek', 'DayOfWeekClaimed']:
    df_new[column + '_sin'] = np.sin(2 * np.pi * df_new[column] / 5)  # Assuming only weekdays
    df_new[column + '_cos'] = np.cos(2 * np.pi * df_new[column] / 5)

# For WeekOfMonth, since it's already numerical
for column in ['WeekOfMonth', 'WeekOfMonthClaimed']:
    df_new[column + '_sin'] = np.sin(2 * np.pi * df_new[column] / 5)
    df_new[column + '_cos'] = np.cos(2 * np.pi * df_new[column] / 5)

# Display the DataFrame
print(df_new['WeekOfMonth_sin'])

0       -2.449294e-16
1       -5.877853e-01
2       -2.449294e-16
3        5.877853e-01
4       -2.449294e-16
             ...     
15415   -9.510565e-01
15416   -2.449294e-16
15417   -2.449294e-16
15418    9.510565e-01
15419    5.877853e-01
Name: WeekOfMonth_sin, Length: 15420, dtype: float64


In [9]:
# label encoding: AccidentArea -> (0,1), Sex -> (0,1), PoliceReportFiled -> (0,1), WitnessPresent -> (0,1), AgentType -> (0,1), Fault -> (0,1) binary

area_to_num = {'Urban': 0, 'Rural': 1}
Sex_to_num = {'Male':0, 'Female':1}
PolicyReport_to_num = {'No':0, 'Yes':1}
WitnessPresent_to_num = {'No':0, 'Yes':1}
Fault_to_num = {'Policy Holder': 0, 'Third Party':1}
AgentType_to_num = {'External':0, 'Internal':1}

df_new['AccidentArea'] = df_new['AccidentArea'].map(area_to_num)
df_new['Sex'] = df_new['Sex'].map(Sex_to_num)
df_new['PoliceReportFiled'] = df_new['PoliceReportFiled'].map(PolicyReport_to_num)
df_new['WitnessPresent'] = df_new['WitnessPresent'].map(WitnessPresent_to_num)
df_new['Fault'] = df_new['Fault'].map(Fault_to_num)
df_new['AgentType'] = df_new['AgentType'].map(AgentType_to_num)

In [10]:
# ordered categorical
# past number of claims
total = len(unique_values)
values = df["PastNumberOfClaims"].unique()

print("=============================================================")
print(f"Unique values : {values}")
print("-------------------------------------------------------------")
print(f"total number of unique values : {total}")


Unique values : ['none' '1' '2 to 4' 'more than 4']
-------------------------------------------------------------
total number of unique values : 13


In [11]:
# Ordered features

ordered_features = ['PastNumberOfClaims', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'NumberOfSuppliments', 'AddressChange_Claim', 'NumberOfCars']

for i in ordered_features:
    unique_values = df[i].unique()
    total = len(unique_values) 
    print("=============================================================")
    print(f"Unique values in '{i}' : {unique_values}")
    print("-------------------------------------------------------------")
    print(f"total number of unique values in '{i}': {total}")

Unique values in 'PastNumberOfClaims' : ['none' '1' '2 to 4' 'more than 4']
-------------------------------------------------------------
total number of unique values in 'PastNumberOfClaims': 4
Unique values in 'AgeOfVehicle' : ['3 years' '6 years' '7 years' 'more than 7' '5 years' 'new' '4 years'
 '2 years']
-------------------------------------------------------------
total number of unique values in 'AgeOfVehicle': 8
Unique values in 'AgeOfPolicyHolder' : ['26 to 30' '31 to 35' '41 to 50' '51 to 65' '21 to 25' '36 to 40'
 '16 to 17' 'over 65' '18 to 20']
-------------------------------------------------------------
total number of unique values in 'AgeOfPolicyHolder': 9
Unique values in 'NumberOfSuppliments' : ['none' 'more than 5' '3 to 5' '1 to 2']
-------------------------------------------------------------
total number of unique values in 'NumberOfSuppliments': 4
Unique values in 'AddressChange_Claim' : ['1 year' 'no change' '4 to 8 years' '2 to 3 years' 'under 6 months']
----

### Ordinal Encoding:

In [12]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(categories=[
    ['none', '1', '2 to 4', 'more than 4'],  # PastNumberOfClaims
    ['new', '2 years', '3 years', '4 years', '5 years', '6 years', '7 years', 'more than 7'],  # AgeOfVehicle
    ['16 to 17', '18 to 20', '21 to 25', '26 to 30', '31 to 35', '36 to 40', '41 to 50', '51 to 65', 'over 65'],  # AgeOfPolicyHolder
    ['none', '1 to 2', '3 to 5', 'more than 5'],  # NumberOfSuppliments
    ['no change', 'under 6 months', '1 year', '2 to 3 years', '4 to 8 years'],  # AddressChange_Claim
    ['1 vehicle', '2 vehicles', '3 to 4', '5 to 8', 'more than 8']  # NumberOfCars
])

df_new[ordered_features] = encoder.fit_transform(df_new[ordered_features])
df_new[ordered_features]

,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,NumberOfSuppliments,AddressChange_Claim,NumberOfCars
0,0.0,2.0,3.0,0.0,2.0,2.0
1,0.0,5.0,4.0,0.0,0.0,0.0
2,1.0,6.0,6.0,0.0,0.0,0.0
3,1.0,7.0,7.0,3.0,0.0,0.0
4,0.0,4.0,4.0,0.0,0.0,0.0
...,...,...,...,...,...,...
15415,2.0,5.0,4.0,0.0,0.0,0.0
15416,3.0,5.0,4.0,3.0,0.0,2.0
15417,3.0,4.0,3.0,1.0,0.0,0.0
15418,0.0,1.0,4.0,3.0,0.0,0.0


Non-ordered features

In [13]:
# PolicyType and PolicyNumber have been excluded 
non_ordered_features = ['Make', 'VehicleCategory', 'BasePolicy', 'MaritalStatus']

# Applying one-hot encoding
df_new = pd.get_dummies(df_new, columns=non_ordered_features)

df_new.head()

,Month,WeekOfMonth,DayOfWeek,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,Age,Fault,...,VehicleCategory_Sedan,VehicleCategory_Sport,VehicleCategory_Utility,BasePolicy_All Perils,BasePolicy_Collision,BasePolicy_Liability,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,MaritalStatus_Widow
0,12,5,3,0,2,1,1,1,21,0,...,0,1,0,0,0,1,0,0,1,0
1,1,3,3,0,1,1,4,0,34,0,...,0,1,0,0,1,0,0,0,1,0
2,10,5,5,0,4,11,2,0,47,0,...,0,1,0,0,1,0,0,1,0,0
3,6,2,6,1,5,7,1,0,65,1,...,0,1,0,0,0,1,0,1,0,0
4,1,5,1,0,2,2,2,1,27,1,...,0,1,0,0,1,0,0,0,1,0


In [14]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 68 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Month                    15420 non-null  int64  
 1   WeekOfMonth              15420 non-null  int64  
 2   DayOfWeek                15420 non-null  int64  
 3   AccidentArea             15420 non-null  int64  
 4   DayOfWeekClaimed         15420 non-null  int64  
 5   MonthClaimed             15420 non-null  int64  
 6   WeekOfMonthClaimed       15420 non-null  int64  
 7   Sex                      15420 non-null  int64  
 8   Age                      15420 non-null  int64  
 9   Fault                    15420 non-null  int64  
 10  VehiclePrice             15420 non-null  object 
 11  FraudFound_P             15420 non-null  int64  
 12  RepNumber                15420 non-null  int64  
 13  Deductible               15420 non-null  int64  
 14  DriverRating          

In [15]:
price_values = df_new['VehiclePrice'].unique()
total = len(price_values)

print("=============================================================")
print(f"Unique values in 'VehiclePrice': {price_values}")
print("-------------------------------------------------------------")
print(f"total number of unique values in 'VehiclePrice': {total}")

accident_values = df_new['Days_Policy_Accident'].unique()
total_accident = len(accident_values)

print("=============================================================")
print(f"Unique values in 'Days_Policy_Accident': {accident_values}")
print("-------------------------------------------------------------")
print(f"total number of unique values in 'Days_Policy_Accident': {total_accident}")

claim_values = df_new['Days_Policy_Claim'].unique()
total_claim = len(claim_values)

print("=============================================================")
print(f"Unique values in 'Days_Policy_Claim': {claim_values}")
print("-------------------------------------------------------------")
print(f"total number of unique values in 'Days_Policy_Claim': {total_claim}")

Unique values in 'VehiclePrice': ['more than 69000' '20000 to 29000' '30000 to 39000' 'less than 20000'
 '40000 to 59000' '60000 to 69000']
-------------------------------------------------------------
total number of unique values in 'VehiclePrice': 6
Unique values in 'Days_Policy_Accident': ['more than 30' '15 to 30' 'none' '1 to 7' '8 to 15']
-------------------------------------------------------------
total number of unique values in 'Days_Policy_Accident': 5
Unique values in 'Days_Policy_Claim': ['more than 30' '15 to 30' '8 to 15' 'none']
-------------------------------------------------------------
total number of unique values in 'Days_Policy_Claim': 4


In [16]:
vehicle_price_order = ['less than 20000', '20000 to 29000', '30000 to 39000', '40000 to 59000', '60000 to 69000', 'more than 69000']
days_policy_accident_order = ['none', '1 to 7', '8 to 15', '15 to 30', 'more than 30']
days_policy_claim_order = ['none', '8 to 15', '15 to 30', 'more than 30']

# Initialize OrdinalEncoder with the defined order
encoder2 = OrdinalEncoder(categories=[vehicle_price_order, days_policy_accident_order, days_policy_claim_order])

# Fitting and transforming the data
# NOTE: Need to ensure no missing categories and all instances match the specified categories
df_new[['VehiclePrice', 'Days_Policy_Accident', 'Days_Policy_Claim']] = encoder2.fit_transform(df_new[['VehiclePrice', 'Days_Policy_Accident', 'Days_Policy_Claim']])
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 68 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Month                    15420 non-null  int64  
 1   WeekOfMonth              15420 non-null  int64  
 2   DayOfWeek                15420 non-null  int64  
 3   AccidentArea             15420 non-null  int64  
 4   DayOfWeekClaimed         15420 non-null  int64  
 5   MonthClaimed             15420 non-null  int64  
 6   WeekOfMonthClaimed       15420 non-null  int64  
 7   Sex                      15420 non-null  int64  
 8   Age                      15420 non-null  int64  
 9   Fault                    15420 non-null  int64  
 10  VehiclePrice             15420 non-null  float64
 11  FraudFound_P             15420 non-null  int64  
 12  RepNumber                15420 non-null  int64  
 13  Deductible               15420 non-null  int64  
 14  DriverRating          

### Deleting features before transforming

In [17]:
cyclic_features = ['Month', 'MonthClaimed', 'DayOfWeek', 'DayOfWeekClaimed', 'WeekOfMonth', 'WeekOfMonthClaimed'] 

# delete cyclic features (using only sin and cos)
df_new = df_new.drop(columns=cyclic_features)

df_new.head()

,AccidentArea,Sex,Age,Fault,VehiclePrice,FraudFound_P,RepNumber,Deductible,DriverRating,Days_Policy_Accident,...,VehicleCategory_Sedan,VehicleCategory_Sport,VehicleCategory_Utility,BasePolicy_All Perils,BasePolicy_Collision,BasePolicy_Liability,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,MaritalStatus_Widow
0,0,1,21,0,5.0,0,12,300,1,4.0,...,0,1,0,0,0,1,0,0,1,0
1,0,0,34,0,5.0,0,15,400,4,4.0,...,0,1,0,0,1,0,0,0,1,0
2,0,0,47,0,5.0,0,7,400,3,4.0,...,0,1,0,0,1,0,0,1,0,0
3,1,0,65,1,1.0,0,4,400,2,4.0,...,0,1,0,0,0,1,0,1,0,0
4,0,1,27,1,5.0,0,3,400,1,4.0,...,0,1,0,0,1,0,0,0,1,0


In [18]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 62 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   AccidentArea             15420 non-null  int64  
 1   Sex                      15420 non-null  int64  
 2   Age                      15420 non-null  int64  
 3   Fault                    15420 non-null  int64  
 4   VehiclePrice             15420 non-null  float64
 5   FraudFound_P             15420 non-null  int64  
 6   RepNumber                15420 non-null  int64  
 7   Deductible               15420 non-null  int64  
 8   DriverRating             15420 non-null  int64  
 9   Days_Policy_Accident     15420 non-null  float64
 10  Days_Policy_Claim        15420 non-null  float64
 11  PastNumberOfClaims       15420 non-null  float64
 12  AgeOfVehicle             15420 non-null  float64
 13  AgeOfPolicyHolder        15420 non-null  float64
 14  PoliceReportFiled     

In [19]:
agent_values = df_new['AgentType'].unique()
total = len(agent_values)

print("=============================================================")
print(f"Unique values in 'AgentType': {agent_values}")
print("-------------------------------------------------------------")
print(f"total number of unique values in 'AgentType': {total}")

Unique values in 'AgentType': [0 1]
-------------------------------------------------------------
total number of unique values in 'AgentType': 2


In [20]:
# stored as insurance data encoded

df_new.to_csv('Vehicle_insurance_encoded.csv', index = False)